In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
import datetime as dat
import matplotlib as plt
import datetime as dt
import xgboost as xgb
%matplotlib inline
sns.set()

In [2]:
matriz = pd.read_csv('../xgb/training_set_sc.csv').set_index('ref_hash')

In [7]:
features = matriz.iloc[:,:-1]
labels = matriz.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=features,label=labels)

/usr/local/lib/python2.7/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python2.7/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [8]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = \
    train_test_split(features, labels, test_size=0.25, random_state=123)

In [20]:
params = {
        'colsample_bytree': [0.3, 0.01],
        'learning_rate': [0.055, 0.001],
        'max_depth': [15, 1],
        'alpha': [10, 0.1],
        'num_boost_round': [50, 1],
        'gamma': [10, 0.1],
        'n_jobs': [10, 1],
        'n_estimators': [89, 1],
        'n_folds': [10, 1],
        }

xg_reg = xgb.XGBRegressor(objective ='reg:linear', max_depth = params['max_depth'][0], learning_rate = params['learning_rate'][0], \
                          n_estimators = params['n_estimators'][0], n_jobs = params['n_jobs'][0], gamma = params['gamma'][0], \
                          colsample_bytree = params['colsample_bytree'][0], alpha = params['alpha'][0], \
                          num_boost_round = params['num_boost_round'][0], n_folds = params['n_folds'][0])

xg_reg.fit(train_features,train_labels)
predicted_labels = xg_reg.predict(test_features)
rmse = np.sqrt(mean_squared_error(test_labels, predicted_labels))
print(rmse)
best_score = rmse
best_try = params.copy()
def probar_sumar(hiperparametro):
    params[hiperparametro][0] += params[hiperparametro][1]
    xg_reg = xgb.XGBRegressor(objective ='reg:linear', max_depth = params['max_depth'][0], learning_rate = params['learning_rate'][0], \
                              n_estimators = params['n_estimators'][0], n_jobs = params['n_jobs'][0], gamma = params['gamma'][0], \
                              colsample_bytree = params['colsample_bytree'][0], alpha = params['alpha'][0], \
                              num_boost_round = params['num_boost_round'][0], n_folds = params['n_folds'][0])
    xg_reg.fit(train_features,train_labels)
    predicted_labels = xg_reg.predict(test_features)
    rmse = np.sqrt(mean_squared_error(test_labels, predicted_labels))
    return rmse
    
def probar_restar(hiperparametro):
    params[hiperparametro][0] -= params[hiperparametro][1]
    xg_reg = xgb.XGBRegressor(objective ='reg:linear', max_depth = params['max_depth'][0], learning_rate = params['learning_rate'][0], \
                              n_estimators = params['n_estimators'][0], n_jobs = params['n_jobs'][0], gamma = params['gamma'][0], \
                              colsample_bytree = params['colsample_bytree'][0], alpha = params['alpha'][0], \
                              num_boost_round = params['num_boost_round'][0], n_folds = params['n_folds'][0])
    xg_reg.fit(train_features,train_labels)
    predicted_labels = xg_reg.predict(test_features)
    rmse = np.sqrt(mean_squared_error(test_labels, predicted_labels))
    return rmse
    
for i in range(2):
    for hiperparam in params:
        mejor = True
        while(mejor):
            score = probar_sumar(hiperparam)
            mejor = ( score < best_score)
            if mejor:
                best_score = score
                print(best_score)
        params[hiperparam][0] -= params[hiperparam][1]
        
        mejor = True
        while(mejor):
            score = probar_restar(hiperparam)
            mejor = ( score < best_score)
            if mejor:
                best_score = score
                print(best_score)
        params[hiperparam][0] += params[hiperparam][1]

params

49077.2275073329
49072.965948734
49069.46759556803
48964.826268336685
48956.85352746344
48949.433273745766
48948.61005886034
48946.74221939751


{'alpha': [10.0, 0.1],
 'colsample_bytree': [0.3, 0.01],
 'gamma': [10.0, 0.1],
 'learning_rate': [0.055, 0.005],
 'max_depth': [15, 1],
 'n_estimators': [89, 1],
 'n_folds': [10, 1],
 'n_jobs': [10, 1],
 'num_boost_round': [50, 1]}